# Forward Rates and Classes - Practical Lesson 4

## Calculating Forward Rates

Last week we wrote a function called `df` for calculating a discount factor at any date, given a set of discount factors each relative to a corresponding pillar date, using log-linear interpolation. Now we want a function to compute forward rates.

The formula to calculate the forward rates can be found exploiting the property that investing at rate $r_1$ for the period $(0, T_1)$ and then *reinvesting* at rate $r_{1,2}$ for the time period $(T_1, T_2)$ is equivalent to invest at rate $r_2$ for the time period $(0, T_2)$ (i.e. no arbitrage condition, two investors shouldn't be able to earn money from arbitraging between different interest periods). That said:

$$(1+r_1 T_1)(1+r_{1,2}(T_2 - T_1)) = 1 + r_2 T_2$$

Solving for $r_{1,2}$ leads to

$$F(T_1, T_2) = r_{1,2} = \frac{1}{T_2-T_1}\Big(\frac{D(T_1)}{D(T_2)} - 1 \Big)~~~~\textrm{(where $D{(T_i)}=\frac{1}{1+r_iT_{i}}$)}$$

In [ ]:
from datetime import date
import numpy, math

today_date = date (2019, 1, 1)

pillar_dates = [date(2019 , 1 ,1), 
                date(2020, 1, 1), 
                date(2021, 10 ,1)]
discount_factors = [1.0, 0.97, 0.72]

def df(d):
    log_discount_factors = [math.log(discount_factor) \
                            for discount_factor in discount_factors]
    pillar_days = [(pillar_date - today_date).days \
                   for pillar_date in pillar_dates]
    d_days = (d - today_date).days
    interpolated_log_discount_factor = \
        numpy.interp(d_days, pillar_days, log_discount_factors)
    
    return math.exp(interpolated_log_discount_factor)

def forward_rate(t1, t2):
    return 365.0/(t2-t1).days * (df(t1) / df(t2) - 1)

forward_rate(date(2019, 2, 1), date(2019, 8, 1))

### 2008 Financial Crisis

Looking at the historical series of the Euribor (6M) rate versus the Eonia Overnight Indexed Swap (OIS-6M) rate over  the time interval 2006-2011 it becomes apparent how before August 2007 the two rates display strictly overlapping trends differing of no more than 6 bps. 

![](credit_crunch.png)

In August 2007 however we observe a sudden increase of the Euribor rate and a simultaneous decrease of the OIS rate  that leads to the explosion of the corresponding basis spread, touching the peak of 222 bps in October 2008, when Lehman Brothers filed for bankruptcy protection. Successively the basis has sensibly reduced and stabilized between  40 bps and 60 bps (notice that the pre-crisis level has never been recovered). The same effect is observed for other similar couples, e.g. Euribor 3M vs OIS 3M. 

The reason of the abrupt divergence between the Euribor and OIS rates can be explained by considering both the monetary policy decisions adopted by international authorities in response to the financial turmoil, and the impact of the credit crunch on the credit and liquidity risk perception of the market, coupled with the different financial meaning and dynamics of these rates. 

* The Euribor rate is the reference rate for over-the-counter (OTC) transactions in the  Euro  area. It is defined as “the rate at which Euro interbank Deposits are being offered within the EMU zone by one prime bank to another at 11:00 a.m. Brussels time". The rate fixings for a strip of 15 maturities, ranging from one day to one year, are constructed as the average of the rates submitted (excluding  the  highest and lowest 15% tails) by  a  panel  of banks 42 banks, selected among the EU banks with the highest volume of business in the Euro zone money markets, plus some large international bank from non-EU countries with important  euro  zone  operations. **Thus, Euribor rates reflect the average cost of  funding of banks in the interbank market at each given maturity. During the crisis the  solvency and solidity of the whole financial sector was brought into question and the credit and liquidity risk and premia associated to interbank counterparties sharply increased.** The Euribor rates immediately reflected these dynamics and raise to their highest values over more than 10 years. As seen in the plot above, the Euribor 6M rate suddenly increased on August 2007 and reached 5.49% on 10th October 2008. 
* The Eonia rate is the reference rate for overnight OTC transactions in the Euro area. It is constructed as the average rate of the overnight transactions (one day maturity deposits) executed during a given business day by a panel of banks on the interbank money market, weighted with the corresponding transaction volumes. **The Eonia Contribution Panel  coincides with the Euribor Contribution Panel, thus Eonia rate includes information on the  short term (overnight) liquidity expectations of banks in the Euro money market. It is also used by the European Central Bank (ECB) as a method of effecting and observing the  transmission of its monetary policy actions. During the crisis the central banks were  mainly concerned about restabilising the level of liquidity in the market, thus they reduced the level of the official rates.** Furthermore, the daily tenor of the Eonia rate makes negligible the credit and liquidity risks reflected on it: for this reason the OIS rates are considered the best proxies available in the market for the risk-free rate.

As a practial result, after the 2008 financial crisis, it is not possible anymore to use a single discount curve to correctly price forward rates of all tenors. For example, if we want to calculate the net present value of a forward 6-month libor coupon, we need to simultaneously use two different discount curves:

* the 6-month libor curve for determining the forward rate
* the EONIA curve for discounting the expected cash flow

Essentially we are now going to explore how to implement the following calculation:

$$\mathrm{NPV} = D_{\mathrm{EONIA}}(T_1) \times \frac{1}{T_2-T_1}\Big(\frac{D_{\mathrm{LIBOR}}(T_1)}{D_{\mathrm{LIBOR}}(T_2)} - 1 \Big)$$

In the next we will develop a class called `DiscountCurve` which helps us in handling discount curves in our studies.

#### Digression
An interactive session (e.g notebook or interactive shell) is great for quick testing and 
exploratory use, but once you have some code (i.e. functions or classes) which you'd like to reuse often, rather than copy/pasting it every time you need it, you can save it in a `.py` file and use it from your session 
(i.e. you can create your own library).
These work just like the modules we have been importing up to now, except they're not written by us...

Today we're going to start writing a module called **finmarkets**, and over the course of the remaining lessons we'll add functionality related to the theory we study **(consider that you will be asked to use as much as possible this module for your final project !).**

According to your preferred way of working there are different instructions to write a module:

* take a look at this video (https://www.youtube.com/watch?v=AqCl65wxikw) for an example of how using your own module is done for *Jupyter notebook*;
* otherwise you can open text editor (Notepad in Windows) or a `python` IDE like `Pycharm` and write your code there; remember then to save the resulting file in the same directory from which you would like to import it. 

So to start our financial module let's create a new file called `finmarkets.py` and code the `DiscountCurve` class there. As for any other module we will be able to import it and use its classes and functions.

This class will keep pillar dates and pillar discount factors as attributes and will have methods for calculating discount factors and forward rates at arbitrary dates.

In [ ]:
import math, numpy as np
from datetime import date

class DiscountCurve:

    # the special __init__ method defines 
    # how to construct instances of the class
    def __init__(self, today, pillar_dates, discount_factors):
        # we just store the arguments as attributes of the instance
        self.today = today
        self.pillar_dates = pillar_dates
        self.discount_factors = discount_factors

    # calculates a discount factor at an arbitrary 
    #value date using the data stored in the instance
    def df(self, d):
        # these remain local variables, 
        # i.e. they are only available within the function.
        # to read (or write) instance attributes, 
        # you always need to use the self. syntax
        log_discount_factors = \
          [math.log(discount_factor) 
           for discount_factor in self.discount_factors]
        pillar_days = [(pillar_date - self.today).days 
                       for pillar_date in self.pillar_dates]
        d_days = (d - self.today).days
        interpolated_log_discount_factor = \
            np.interp(d_days, pillar_days, log_discount_factors)
        return math.exp(interpolated_log_discount_factor)

    # calculates a forward libor rate based on the discount 
    # curve data stored in the instance
    def forward_rate(self, d1, d2):
        # we use the df method of the current instance to calculate
        # the forward rate
        return (self.df(d1) / self.df(d2) - 1.0) * \
                (365.0 / ((d2 - d1).days))

### Exercise

As an exercise try to complete the development of the class implementing two methods `df` and `forward_rate` to compute discount factors and forward rates at arbitrary dates. Finally save the resulting class in the finmarket.py module as the first step in building our pythonic financial libray.
**Hint**: try to reuse the interpolation code we wrote before.

In the meantime I have already done it, so let's make some test to check that is actually working.

In [3]:
from datetime import date
from finmarkets import DiscountCurve

# build the EONIA curve object
# n.b. here we use the 'parameter=argument' syntax 
# (today=..., pillar_dates=...)
eonia_curve = DiscountCurve(today=date(2020, 10, 1),
                            pillar_dates=[date(2020, 10, 1), 
                                          date(2021, 10, 1), 
                                          date(2022, 10, 1)],
                            discount_factors=[1.0, 0.95, 0.8])

# build the Libor curve object
libor_curve = DiscountCurve(today=date(2020, 10, 1),
                            pillar_dates=[date(2020, 10, 1), 
                                          date(2021, 4, 1), 
                                          date(2021, 10, 1)],
                            discount_factors=[1.0, 0.98, 0.82])

# Let's compute the discount factor of the two curves
# on the 2021-5-1
print ("Discount factor EONIA at {}: {:.3f}".format(date(2021, 5, 1), eonia_curve.df(date(2021, 5, 1))))
print ("Discount factor LIBOR at {}: {:.3f}".format(date(2021, 5, 1), libor_curve.df(date(2021, 5, 1))))

Discount factor EONIA at 2021-05-01: 0.971
Discount factor LIBOR at 2021-05-01: 0.952


Let's compute the 6m forward rate at 2021-04-01.

In [7]:
from dateutil.relativedelta import relativedelta

t1 = date(2021, 4, 1)
t2 = t1 + relativedelta(months=6)

print ("6m forward rate at {}: {:.3f}".format(t1, libor_curve.forward_libor(t1, t2)))

6m forward rate at 2021-04-01: 0.389


Finally compute the NPV of a 6m forward libro coupon, both in the current and pre-2008 way:

In [11]:
npv = eonia_curve.df(t1) * libor_curve.forward_libor(t1, t2)
npv_pre_2008 = libor_curve.df(t1) * libor_curve.forward_libor(t1, t2)

print ("NPV current way: {:.3f}".format(npv))
print ("NPV pre-2008 way: {:.3f}".format(npv_pre_2008))

NPV current way: 0.379
NPV pre-2008 way: 0.381
